# 19 - Python Concurrency

[Source Code from the Book](https://github.com/fluentpython/example-code-2e/tree/master/19-concurrency )

## Spinner with Threads

Below, we have 2 functions (`slow` and `spin`).  
Start a function (`slow`) that blocks for 3 seconds, and this is to simulate a computing task, like a slow API call over the network.  
The `spin` function makes an animated spinner displaying each character in the string `\|/-` in the same screen position. When the `slow` computation finishes, the line with the spinner is cleared and the result is shown (42).  
The spinner will spin simultaneously while we wait for `slow` to finish => we are doing 2 things at the same time.  

In [24]:
# spinner_thread.py

import itertools
import time
from threading import Thread, Event


def spin(msg: str, done: Event) -> None:  # This function will run in a separate thread. Event is used to synchronize threads 
    for char in itertools.cycle(r'\|/-'):  # This is an infinite loop because itertools.cycle yields one character at a time
        status = f'\r{char} {msg}'  # Trick for text-mode animation: move the cursor back to the start of the line with the carriage return ASCII control character ('\r').
        print(status, end='', flush=True)
        if done.wait(.1):  # .wait returns True when the event is set() by another thread; if the timeout elapses, it returns False
            break  # Exit the infinite loop
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')  # Clear the status line by overwriting with spaces and moving the cursor back to the beginning.


def slow() -> int:
    time.sleep(3)  # slow() will be called by the main thread. Imagine this is a slow API call over the network.
    return 42


def supervisor() -> int:  # supervisor will return the result of slow
    done = Event()  # Event is used to coordinate the main thread and spinner thread
    spinner = Thread(target=spin, args=('thinking!', done))  # Run a new thread with the spin function as target
    print(f'spinner object: {spinner}')  # the spinner object will be <Thread(Thread-1, initial)>, where initial means the thread is not started yet
    spinner.start()  # Start the spinner thread.
    result = slow()  # Call slow, which blocks the main thread. Meanwhile, the spinner thread is running.
    done.set()  # Set the Event flag to True; this will terminate the for loop inside the spin function
    spinner.join()  # Wait until the spinner thread finishes.
    return result


def main() -> None:
    result = supervisor()  # Run the supervisor function.
    print(f'Answer: {result}')


main()

spinner object: <Thread(Thread-7 (spin), initial)>
Answer: 42  


&#128212; `time.sleep()` blocks the calling thread but releases the GIL, allowing other Python threads to run.

## Spinner with Processes

The `multiprocessing` package supports running concurrent tasks in _separate Python processes_ instead of threads. When you create a `multiprocessing.Process` instance, a whole new Python interpreter is started as a child process in the background. Since each python interpreter has its own GIL, this allows the program to use all available CPU cores.

In [26]:
# spinner_proc.py

import itertools
import time
from multiprocessing import Process, Event  
from multiprocessing import synchronize     


def spin(msg: str, done: synchronize.Event) -> None:  # difference to the thread version here
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def slow() -> int:
    time.sleep(3)
    return 42


def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))  # difference to the thread version here
    print(f'spinner object: {spinner}')          
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result


def main() -> None:
    result = supervisor()
    print(f'Answer: {result}')


main()

spinner object: <Process name='Process-4' parent=67182 initial>
Answer: 42  


`67182` is the process ID of the Python instance running `spinner_proc.py`

&#128212; The basic API of `thread` and `multiprocessing` are similar, but their implementation is very different, and `multiprocessing` has a much larger API to handle the added complexity of multiprocess programming, e.g. how to communicate between processes by serializing and deserializing objects

## Spinner with Coroutines

Python coroutines usually run within a single thread under the supervision of an event loop, also in the same thread. Coroutines support cooperative multitasking: each coroutine must explicitly cede control with the `yield` or `await` keyword, so that another may proceed concurrently (but not in parallel)

In the code below in `spinner_async.py`, we only have `main` as a function. Others are `coroutines`

In [25]:
!python spinner_async.py

spinner object: <Task pending name='Task-2' coro=<spin() running at /home/dk/Desktop/projects/programming/fluent-python/19-concurrency/spinner_async.py:5>>
END OF SPIN 
Answer: 42


### Spinner with `async` and `time.sleep`  

In this experiement, we use `time.sleep` instead of `asyncio.sleep`, hence pause the whole program and we will not be able to see the spinner at all

In [22]:
!python spinner_async_experiment.py

spinner object: <Task pending name='Task-2' coro=<spin() running at /home/dk/Desktop/projects/programming/fluent-python/19-concurrency/spinner_async_experiment.py:6>>
Answer: 42


&#128212; When `time.sleep()` is called, it will block the entire execution of the script and it will be put on hold, just frozen, doing nothing. But when you call await `asyncio.sleep()`, it will ask the asyncio event loop to run something else while your `await` statement finishes its execution.

## Spinner using Threads and Coroutines: Differences and Similarities 

<table>
<tr>
<th> threaded supervisor function in `spinner_thread.py` </th>
<th> the asynchronous supervisor coroutine in `spinner_async.py` </th>
</tr>
<tr>
<td>

```python
def supervisor() -> int: 
    done = Event()  
    spinner = Thread(target=spin, args=('thinking!', done))  
    print(f'spinner object: {spinner}') 
    spinner.start()  
    result = slow() 
    done.set()  
    spinner.join()  
    return result
```

</td>
<td>

```python
async def supervisor() -> int:
    spinner = asyncio.create_task(spin('thinking!'))
    print('spinner object:', spinner)
    result = await slow()
    spinner.cancel()
    return result
```

• An `asyncio.Task` is roughly the equivalent of a `threading.Thread`  
• A `Task` drives a coroutine object, and a `Thread` invokes a callable.  
• A coroutine yields control explicitly with the `await` keyword.  
• You don’t instantiate Task objects yourself, you get them by passing a coroutine to `asyncio.create_task(…)`.  
• When `asyncio.create_task(…)` returns a `Task` object, it is already scheduled to run, but a `Thread` instance must be explicitly told to run by calling its start method.  
• In the threaded `supervisor`, `slow` is a plain function and is directly invoked by the main thread (it uses `time.sleep(...)`). In the asynchronous `supervisor`, `slow` is a coroutine driven by `await` (it uses `asyncio.await`).  
• There’s no API to terminate a thread from the outside; instead, you must send a signal (the `Event.set()` function). For tasks, there is the `Task.cancel()` instance method, which raises `CancelledError` at the `await` expression where the coroutine body is currently suspended.  
• The supervisor coroutine must be started with `asyncio.run` in the `main` function.

&#128212; With thread, the scheduler can interrupt a thread at any time. You must remember to hold locks to protect the critical sections of your program, to avoid getting interrupted in the middle of a multistep operation, which could leave data in an invalid state.

With coroutines, your code is protected against interruption by default. Coroutines are “synchronized” by definition: only one of them is running at any time. You must explicitly `await` to let the rest of the program run